In [51]:
import sys
sys.path.append('../scripts')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import helpers as hp
import pickle as pkl
import os

from keras.layers import Input, Dense, LSTM, Embedding, Merge
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adadelta
from keras.callbacks import Callback
from keras import backend as K

from sklearn.metrics import f1_score, precision_score, recall_score,\
                            average_precision_score, roc_auc_score,\
                            roc_curve, precision_recall_curve, confusion_matrix,\
                            accuracy_score

from IPython.core.interactiveshell import InteractiveShell
from matplotlib import rcParams
from importlib import reload

rcParams['font.family'] = 'serif'
rcParams['font.serif'] = 'times new roman'

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Data Preprocessing

In [2]:
with open('../data/embeddings/glove-300.map', 'rb') as f:
    map = pkl.load(f)

data_dir = os.path.join('..','data')
source_dir = os.path.join(data_dir,'split','amazon-google')
data = hp.load_data(source_dir)

datasets = ['train_1', 'val_1', 'test_1', 'train_2', 'val_2', 'test_2']

# drop columns that do not contain embeddings
data['train_1'] = data['train_1'].drop(['id1', 'price'], axis = 'columns')
data['train_2'] = data['train_2'].drop(['id2', 'price'], axis = 'columns')
data['val_1'] = data['val_1'].drop(['id1', 'price'], axis = 'columns')
data['val_2'] = data['val_2'].drop(['id2', 'price'], axis = 'columns')
data['test_1'] = data['test_1'].drop(['id1', 'price'], axis = 'columns')
data['test_2'] = data['test_2'].drop(['id2', 'price'], axis = 'columns')


# concatenate all embedding columns so each record becomes one giant list
def concat_columns(x):
    x = x['title'] + x['description'] + x['manufacturer']
    return x

for df_name in datasets:
    data[df_name] = data[df_name].apply(concat_columns, axis='columns')
    
# pad all sequences with less than 500 indices with zeros
# limit all sequences to 500 indices
maxlen = 500
for df_name in datasets:
    data[df_name] = pad_sequences(data[df_name], maxlen=maxlen)

# Build Model

In [5]:
glove_matrix = np.load(os.path.join(data_dir, 
                                    'embeddings',
                                    'glove-300.matrix.npy'))

In [6]:
n_hidden = 50
gradient_clipping_norm = 1.25

input_left = Input(shape=(maxlen,))
input_right = Input(shape=(maxlen,))

embedding_layer = Embedding(glove_matrix.shape[0],
                            glove_matrix.shape[1],
                            weights=[glove_matrix],
                            input_length=maxlen,
                            trainable=False)
embedded_left = embedding_layer(input_left)
embedded_right = embedding_layer(input_right)

lstm_layer = LSTM(n_hidden)
output_left = lstm_layer(embedded_right)
output_right = lstm_layer(embedded_left)

def distance_metric(x1, x2):
    l1 = -K.sum(K.abs(x1 - x2), axis=1, keepdims=True)
    return K.exp(l1)

merge_layer = Merge(mode=lambda x: distance_metric(x[0], x[1]),
                    output_shape=lambda x: (x[0][0], 1))
distance = merge_layer([output_left, output_right])

malstm = Model([input_left, input_right], [distance])

/home/paperspace/anaconda3/envs/python-36/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [7]:
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error',
               optimizer=optimizer,
               metrics=['accuracy'])

In [43]:
batch_size = 2048
epochs = 50

malstm_trained = malstm.fit([data['train_1'], data['train_2']],
                            data['train_y'],
                            batch_size=batch_size,
                            epochs=epochs,
                            class_weight={0:1, 1:5},
                            shuffle=True)
history.append(malstm_trained.history)

Epoch 1/50
10400/10400 [==============================] - 14s 1ms/step - loss: 0.1806 - acc: 0.9373
Epoch 2/50
10400/10400 [==============================] - 13s 1ms/step - loss: 0.1761 - acc: 0.9400
Epoch 3/50
10400/10400 [==============================] - 14s 1ms/step - loss: 0.1775 - acc: 0.9377
Epoch 4/50
10400/10400 [==============================] - 13s 1ms/step - loss: 0.1714 - acc: 0.9423
Epoch 5/50
10400/10400 [==============================] - 13s 1ms/step - loss: 0.1724 - acc: 0.9408
Epoch 6/50
10400/10400 [==============================] - 13s 1ms/step - loss: 0.1716 - acc: 0.9415
Epoch 7/50
10400/10400 [==============================] - 13s 1ms/step - loss: 0.1701 - acc: 0.9412
Epoch 8/50
10400/10400 [==============================] - 13s 1ms/step - loss: 0.1709 - acc: 0.9387
Epoch 9/50
10400/10400 [==============================] - 13s 1ms/step - loss: 0.1680 - acc: 0.9427
Epoch 10/50
10400/10400 [==============================] - 13s 1ms/step - loss: 0.1661 - acc: 0.9430

In [44]:
y_train_pred = malstm.predict([data['train_1'], data['train_2']])
confusion_matrix(data['train_y'], y_train_pred >= 0.5)

array([[9093,  259],
       [ 287,  761]])

In [45]:
y_val_pred = malstm.predict([data['val_1'], data['val_2']])

array([[1138,   49],
       [  75,   38]])

In [78]:
threshold = 0.5
print(confusion_matrix(data['train_y'], y_train_pred >= threshold))
print(accuracy_score(data['train_y'], y_train_pred >= threshold))
print(f1_score(data['train_y'], y_train_pred >= threshold))

[[9093  259]
 [ 287  761]]
0.9475
0.7359767891682786


In [79]:
threshold = 0.5
print(accuracy_score(data['val_y'], y_val_pred >= threshold))
print(confusion_matrix(data['val_y'], y_val_pred >= threshold))
print(f1_score(data['val_y'], y_val_pred >= threshold))

0.9046153846153846
[[1138   49]
 [  75   38]]
0.38000000000000006
